In [1]:
import pymc as pm
import arviz as az
import numpy as np

# 간단한 모델 예제
with pm.Model() as model:
    theta = pm.Normal("theta", mu=0, sigma=1)
    trace = pm.sample(1000, tune=500)

# 결과 요약
print(az.summary(trace))


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [theta]


Output()

Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 23 seconds.


        mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  ess_tail  \
theta -0.013  0.992  -1.828    1.889      0.025    0.018    1525.0    2841.0   

       r_hat  
theta    1.0  


In [4]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from lifelines.utils import concordance_index

# 데이터 로드 및 전처리
data = pd.read_csv("./colon_gu.csv")
duration_col = "stime"
event_col = "event_inc"

# 지역 변수 통합 (가장 큰 값의 인덱스를 그룹으로 설정)
data["region"] = data[["gu_1", "gu_2", "gu_3", "gu_4", "gu_5", 
                       "gu_6", "gu_7", "gu_8", "gu_9", "gu_10", 
                       "gu_11", "gu_12", "gu_13", "gu_14", "gu_15"]].idxmax(axis=1)

# 지역 이름을 숫자로 변환 및 0부터 시작하도록 매핑
unique_regions = {region: idx for idx, region in enumerate(data["region"].unique())}
data["region"] = data["region"].map(unique_regions)

# 생존 시간 값이 0 이하인 경우 수정
data[duration_col] = data[duration_col].apply(lambda x: 0.01 if x <= 0 else x)

# 전체 고유 지역 수 계산
n_regions = len(unique_regions)

# Cross-validation 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)
c_indices = []

for train_index, test_index in kf.split(data):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    # PyMC를 사용한 Frailty 모델 정의
    with pm.Model() as frailty_model:
        # Frailty (전체 지역 고유 수에 기반)
        frailty = pm.Normal("frailty", mu=0, sigma=1, shape=n_regions)
        
        # 기저 위험률 (Baseline Hazard)
        baseline_hazard = pm.Normal("baseline_hazard", mu=0, sigma=1)
        
        # 선형 예측값
        log_hazard = baseline_hazard + frailty[train_data["region"].values]
        
        # 위험률 (Hazard)
        hazard = pm.Deterministic("hazard", pm.math.exp(log_hazard))
        
        # 생존 시간 모델링 (Exponential distribution)
        observed = pm.Exponential("observed", lam=hazard, observed=train_data[duration_col])
        
        # 샘플링
        trace = pm.sample(1000, tune=500, return_inferencedata=False, progressbar=False)
    
    # 테스트 데이터 예측
    frailty_test = trace["frailty"].mean(axis=0)[test_data["region"].values]
    baseline_hazard_test = trace["baseline_hazard"].mean()
    log_hazard_test = baseline_hazard_test + frailty_test
    hazard_test = np.exp(log_hazard_test)

    # C-index 계산
    c_index = concordance_index(test_data[duration_col], -hazard_test, test_data[event_col])
    c_indices.append(c_index)

# 결과 출력
print(f"Cross-validated C-index: {np.mean(c_indices):.3f}")
print(f"C-index for each fold: {c_indices}")


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [frailty, baseline_hazard]
Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 81 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [frailty, baseline_hazard]
Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 93 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for so

Cross-validated C-index: 0.516
C-index for each fold: [0.520408629755397, 0.520460795680435, 0.5164896905437558, 0.5157269106023675, 0.5050184801835526]


In [5]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from lifelines.utils import concordance_index

# 데이터 로드 및 전처리
data = pd.read_csv("./colon_gu.csv")
duration_col = "stime"
event_col = "event_inc"

# 지역 변수 통합 (가장 큰 값의 인덱스를 그룹으로 설정)
data["region"] = data[["gu_1", "gu_2", "gu_3", "gu_4", "gu_5", 
                       "gu_6", "gu_7", "gu_8", "gu_9", "gu_10", 
                       "gu_11", "gu_12", "gu_13", "gu_14", "gu_15"]].idxmax(axis=1)

# 지역 이름을 숫자로 변환 및 0부터 시작하도록 매핑
unique_regions = {region: idx for idx, region in enumerate(data["region"].unique())}
data["region"] = data["region"].map(unique_regions)

# 생존 시간 값이 0 이하인 경우 수정
data[duration_col] = data[duration_col].apply(lambda x: 0.01 if x <= 0 else x)

# 전체 고유 지역 수 계산
n_regions = len(unique_regions)

# Cross-validation 설정
kf = KFold(n_splits=5, shuffle=True, random_state=42)
c_indices = []

for train_index, test_index in kf.split(data):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    # PyMC를 사용한 Frailty 모델 정의
    with pm.Model() as frailty_model:
        # 감마 분포 기반 Frailty (양수 값으로 제한)
        frailty = pm.Gamma("frailty", alpha=2.0, beta=1.0, shape=n_regions)
        
        # 기저 위험률 (Baseline Hazard)
        baseline_hazard = pm.Normal("baseline_hazard", mu=0, sigma=1)
        
        # 선형 예측값
        log_hazard = baseline_hazard + pm.math.log(frailty[train_data["region"].values])  # 감마 분포는 로그 공간에서 사용
        
        # 위험률 (Hazard)
        hazard = pm.Deterministic("hazard", pm.math.exp(log_hazard))
        
        # 생존 시간 모델링 (Exponential distribution)
        observed = pm.Exponential("observed", lam=hazard, observed=train_data[duration_col])
        
        # 샘플링
        trace = pm.sample(1000, tune=500, return_inferencedata=False, progressbar=False)
    
    # 테스트 데이터 예측
    frailty_test = trace["frailty"].mean(axis=0)[test_data["region"].values]
    baseline_hazard_test = trace["baseline_hazard"].mean()
    log_hazard_test = baseline_hazard_test + np.log(frailty_test)
    hazard_test = np.exp(log_hazard_test)

    # C-index 계산
    c_index = concordance_index(test_data[duration_col], -hazard_test, test_data[event_col])
    c_indices.append(c_index)

# 결과 출력
print(f"Cross-validated C-index: {np.mean(c_indices):.3f}")
print(f"C-index for each fold: {c_indices}")


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [frailty, baseline_hazard]
Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 95 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [frailty, baseline_hazard]
Sampling 4 chains for 500 tune and 1_000 draw iterations (2_000 + 4_000 draws total) took 108 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for s

Cross-validated C-index: 0.516
C-index for each fold: [0.5204601407738901, 0.520460795680435, 0.5164896905437558, 0.5160581412478847, 0.5050184801835526]
